# Original model prediction

In [6]:
import os 
import numpy as np
os.chdir("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/tcr/")
import model_utils 
tcrbert_trb_cls = model_utils.load_classification_pipeline("wukevin/tcr-bert", device=0)


In [8]:
import pandas as pd
def prediction(data_path,result_path):
    data=pd.read_csv(data_path)
    formatted_strings = [' '.join(list(s)) for s in (data['CDR3B'].tolist())]
    pred = model_utils.reformat_classification_pipeline_preds(tcrbert_trb_cls(formatted_strings))
    unique_epitopes = pd.unique(data['Epitope'])
    df_filtered = pred[unique_epitopes]
    result = pd.concat([data, df_filtered], axis=1)
    result['y_prob'] = result.apply(lambda row: row[row['Epitope']] if row['Epitope'] in result.columns else None, axis=1)
    result=result[['Epitope','CDR3B','antigen_species','Affinity','y_prob']]
    result = result.rename(columns={'Affinity': 'y_true'})
    result['y_pred'] = result['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
    result.to_csv(result_path+'probability.csv')

In [9]:

 name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
        data_path= "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/"+i+'_'+j+'.csv'
        result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/result/Original/seen/seven/"+i+'_'+j
        prediction(data_path,result_path)       
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
        data_path= "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/"+i+'_'+j+'.csv'
        result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/result/Original/seen/three/"+i+'_'+j
        prediction(data_path,result_path)

# Model retraining

In [8]:
import os
import sys
import logging
import argparse
import json
from joblib import dump
from typing import *
import numpy as np
import pandas as pd
import sklearn
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.base import BaseEstimator
from sklearn import metrics
import git 

import sys
sys.path.append("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/tcr/")
import featurization as ft
import data_loader as dl
import model_utils
import canonical_models as models
logging.basicConfig(level=logging.INFO)


def get_model(keyword: str, n_components: int) -> BaseEstimator:
    """
    Return a sklearn type model given a keyword
    """
    if keyword == "pcasvm":
        cls = models.ModelOnPCA(
            SVC, n_components=n_components, probability=True, kernel="rbf"
        )
    elif keyword == "svm":
        cls = SVC(probability=True, kernel="rbf", random_state=6489)
    elif keyword == "lr":
        cls = LogisticRegression(penalty="l2", solver="liblinear")
    elif keyword == "gpc":
        cls = GaussianProcessClassifier()
    else:
        raise ValueError(f"Unrecognized classifier: {keyword}")
    logging.info(f"Classifier {cls}")
    return cls


        
def train_main(trainfile_path,testfile_path,save_model_path,result_path):
    import torch
    import pandas as pd
    import numpy as np
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score,auc,roc_curve
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    train = pd.read_csv(trainfile_path)
    test = pd.read_csv(testfile_path)
    epitope = pd.unique(train['Epitope'])
    results_list = []
    probab_list=[]
    for i in epitope:
        df_train = train[train['Epitope'] == i].sample(frac=1, random_state=42)  # Filter the dataframe for current epitope
        train_seqs = df_train['CDR3B'].tolist()
        train_labels = df_train['Affinity'].tolist()
       
        df_test = test[test['Epitope'] == i].sample(frac=1, random_state=42)  # Filter the dataframe for current epitope
        test_seqs= df_test['CDR3B'].tolist()
        test_labels = df_test['Affinity'].tolist()
        transformer="wukevin/tcr-bert"
        layer=-1
        train_embed = model_utils.get_transformer_embeddings(
                model_dir=transformer,
                seqs=train_seqs,
                layers=[layer],
                method="mean",
                device=device,
            )
        test_embed = model_utils.get_transformer_embeddings(
                model_dir=transformer,
                seqs=test_seqs,
                layers=[layer],
                method="mean",
                device=device,
            )
        numpcs=50
        classifier="svm"
        cls = get_model(classifier, numpcs)
        cls.fit(train_embed, train_labels)
        model_path = save_model_path + '_' + i + '_model.pt'
        torch.save(cls, model_path)

        test_preds = cls.predict_proba(test_embed)[:, 1]
        y_pred = test_preds.round()
        test_labels = np.array(test_labels)
        probab= { 'Epitope': [i] * len(df_test['CDR3B']),'CDR3B': df_test['CDR3B'],'y_true': test_labels, 'y_pred': y_pred,'y_prob': test_preds}
        probab_list.append(probab)   
        probability = pd.DataFrame(probab_list)
        probability = pd.DataFrame(probability)
        probability = probability.apply(pd.Series.explode)
        probability.to_csv(result_path+'probability.csv')

        

# pair50

In [ ]:
        
import os
GPU_NUMBER = [0]
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(s) for s in GPU_NUMBER])
os.environ["NCCL_DEBUG"] = "INFO" 

database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database: 
    for j in name:
        for k in name1:
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+j+"train.csv"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+"1_1test.csv"
            save_model_path="./model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="./result/pair50/seen/test/"+i+"/"+k+'_'+j
            train_main(trainfile_path,testfile_path,save_model_path,result_path)  
            
            
database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+j+"train.csv"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+"1_1test.csv"
            save_model_path="./model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="./result/pair50/seen/test/"+i+"/"+k+'_'+j
            train_main(trainfile_path,testfile_path,save_model_path,result_path)  

# pair300

In [10]:
import os
GPU_NUMBER = [0]
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(s) for s in GPU_NUMBER])
os.environ["NCCL_DEBUG"] = "INFO" 

pair=['more300','300','200','100','10']          
database=['healthy','patient','Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database: 
    for j in name:
        for k in name1:
            for l in pair:
                trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j+"train.csv"
                testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/more300/"+i+"/"+k+'_'+"1_1test.csv"
                save_model_path="./model/Retrain/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                result_path="./result/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                train_main(trainfile_path,testfile_path,save_model_path,result_path)  
            


/home/luyanping/miniconda3/envs/tcrbert/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3382: DtypeWarning: Columns (5,6,7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializ

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:Classifier SVC(probability=True, random_state=6489)
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on 

INFO:root:Classifier SVC(probability=True, random_state=6489)
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on 

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:Classifier SVC(probability=True, random_state=6489)
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on 

INFO:root:Classifier SVC(probability=True, random_state=6489)
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on 

...

INFO:root:Classifier SVC(probability=True, random_state=6489)
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on 

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:Classifier SVC(probability=True, random_state=6489)
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on 

INFO:root:Classifier SVC(probability=True, random_state=6489)
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on 

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:Classifier SVC(probability=True, random_state=6489)
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on 

INFO:root:Classifier SVC(probability=True, random_state=6489)
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on 

In [2]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score, auc, roc_curve

def load_model(model_path):
    """
    Load a pre-trained model from the specified path.
    """
    return torch.load(model_path)

def validation_main(testfile_path, modelfile_path, result_path):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    test = pd.read_csv(testfile_path)
    epitope = pd.unique(test['Epitope'])
    results_list = []
    probab_list = []
    for i in epitope:
        model_path = modelfile_path + '_' + i + '_model.pt'
        cls = load_model(model_path)
        df_test = test[test['Epitope'] == i].sample(frac=1, random_state=42)
        test_seqs = df_test['CDR3B'].tolist()
        test_labels = df_test['Affinity'].tolist()
        transformer = "wukevin/tcr-bert"
        layer = -1
        test_embed = model_utils.get_transformer_embeddings(
            model_dir=transformer,
            seqs=test_seqs,
            layers=[layer],
            method="mean",
            device=device,
        )

        test_preds = cls.predict_proba(test_embed)[:, 1]
        y_pred = test_preds.round()
        test_labels = np.array(test_labels)
        probab = {
            'Epitope': [i] * len(df_test['CDR3B']),
            'CDR3B': df_test['CDR3B'],
            'y_true': test_labels,
            'y_pred': y_pred,
            'y_prob': test_preds
        }
        probab_list.append(probab)
    probability = pd.DataFrame(probab_list)
    probability = pd.DataFrame(probability)
    probability = probability.apply(pd.Series.explode)
    probability.to_csv(result_path + 'probability.csv', index=False)


# Validaiton

In [3]:
import os
GPU_NUMBER = [1]
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(s) for s in GPU_NUMBER])
os.environ["NCCL_DEBUG"] = "INFO" 


In [7]:
import os 
import numpy as np
os.chdir("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert")

database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            testfile_path= index_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+"1_1Validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="./result/pair50/seen/validation/"+i+"/"+k+'_'+j
            validation_main(testfile_path,modelfile_path,result_path)

            
database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            testfile_path= index_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+"1_1Validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="./result/pair50/seen/validation/"+i+"/"+k+'_'+j
            validation_main(testfile_path,modelfile_path,result_path)
            

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

...

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at wukevin/tcr-bert were not used when initializing BertModel: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

# Top5 parameter adjustment

In [2]:
import os
import sys
import logging
import argparse
import json
from joblib import dump
from typing import *
import numpy as np
import pandas as pd
import sklearn
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.base import BaseEstimator
from sklearn import metrics
import git 

import sys
sys.path.append("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/tcr/")
import featurization as ft
import data_loader as dl
import model_utils
import canonical_models as models
logging.basicConfig(level=logging.INFO)


def get_model(keyword: str, n_components: int) -> BaseEstimator:
    """
    Return a sklearn type model given a keyword
    """
    if keyword == "pcasvm":
        cls = models.ModelOnPCA(
            SVC, n_components=n_components, probability=True, kernel="rbf"
        )
    elif keyword == "svm":
        cls = SVC(probability=True, kernel="rbf", random_state=6489)
    elif keyword == "lr":
        cls = LogisticRegression(penalty="l2", solver="liblinear")
    elif keyword == "gpc":
        cls = GaussianProcessClassifier()
    else:
        raise ValueError(f"Unrecognized classifier: {keyword}")
    logging.info(f"Classifier {cls}")
    return cls


        
def train_main(trainfile_path,testfile_path,result_path):
    import torch
    import pandas as pd
    import numpy as np
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score,auc,roc_curve
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    train = pd.read_csv(trainfile_path)
    test = pd.read_csv(testfile_path)
    epitope = pd.unique(train['Epitope'])
    results_list = []
    probab_list=[]
    for i in epitope:
        df_train = train[train['Epitope'] == i].sample(frac=1, random_state=42)  # Filter the dataframe for current epitope
        train_seqs = df_train['CDR3B'].tolist()
        train_labels = df_train['Affinity'].tolist()
       
        df_test = test[test['Epitope'] == i].sample(frac=1, random_state=42)  # Filter the dataframe for current epitope
        test_seqs= df_test['CDR3B'].tolist()
        test_labels = df_test['Affinity'].tolist()
        transformer="wukevin/tcr-bert"
        layer=-1
        train_embed = model_utils.get_transformer_embeddings(
                model_dir=transformer,
                seqs=train_seqs,
                layers=[layer],
                method="mean",
                device=device,
            )
        test_embed = model_utils.get_transformer_embeddings(
                model_dir=transformer,
                seqs=test_seqs,
                layers=[layer],
                method="mean",
                device=device,
            )
        numpcs=50
        classifier="svm"
        cls = get_model(classifier, numpcs)
        cls.fit(train_embed, train_labels)
        test_preds = cls.predict_proba(test_embed)[:, 1]
        y_pred = test_preds.round()
        test_labels = np.array(test_labels)
        probab= { 'Epitope': [i] * len(df_test['CDR3B']),'CDR3B': df_test['CDR3B'],'y_true': test_labels, 'y_pred': y_pred,'y_prob': test_preds}
        probab_list.append(probab)   
        probability = pd.DataFrame(probab_list)
        probability = pd.DataFrame(probability)
        probability = probability.apply(pd.Series.explode)
        probability.to_csv(result_path+'probability.csv')

        

In [3]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

reap=['reap1','reap2','reap3','reap4','reap5']
name=['top1','top2','top3','top4','top5']
number= ['3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50']
for i in reap:
    for j in name:
        for k in number:
            trainfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+k+'.csv'
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+'test.csv'    
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/result/top5/seen/healthy/"+i+'_'+j+'_'+k
            train_main(trainfile_path, testfile_path,result_path)  

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo

...

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo

INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:root:Classifier SVC(probability=True, random_state=6489)
INFO:roo